In [389]:
import backtrader as bt
import pandas as pd
import ccxt
from datetime import datetime

In [390]:
def fetch_binance_data(symbol='BTC/USDT', timeframe='1d'):
    exchange = ccxt.binance()
    since = exchange.parse8601('2017-01-01T00:00:00Z')
    all_ohlcv = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=1000)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1
    
    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

In [391]:
temp = fetch_binance_data()

# remove the timestamp column
temp = temp.drop(columns=['timestamp'])

# set the datetime column as the index
temp = temp.set_index('datetime')

In [392]:
data = bt.feeds.PandasData(dataname=temp)

In [393]:
import backtrader as bt

class PNLAnalyzer(bt.Analyzer):
    def __init__(self):
        self.monthly_start_value = {}  # 紀錄每月的起始總資產（含浮動盈虧）
        self.current_month = None  

        self.yearly_start_value = {}  # 紀錄每年的起始總資產（含浮動盈虧）
        self.current_year = None

    def next(self):
        current_date = self.strategy.datetime.datetime() # 取得當前日期

        """每月的第一天記錄起始總資產"""
        current_month = current_date.strftime('%Y-%m')

        if self.current_month != current_month:
            self.current_month = current_month
            self.monthly_start_value[current_month] = self.strategy.broker.get_value()  # 包含浮動盈虧

        """每年的第一天記錄起始總資產"""
        current_year = current_date.strftime('%Y')

        if self.current_year != current_year:
            self.current_year = current_year
            self.yearly_start_value[current_year] = self.strategy.broker.get_value() # 包含浮動盈虧

    def stop(self):
        self.monthly_start_value['2099-01-01'] = self.strategy.broker.get_value() # 用來計算當前月份的盈虧
        self.yearly_start_value['2099'] = self.strategy.broker.get_value() # 用來計算當前年份的盈虧

        # 計算每月盈虧
        sorted_months = sorted(self.monthly_start_value.keys())  # 按時間排序
        previous_value = None
        previous_month = None
        monthly_pnl = {}
        for month in sorted_months:
            current_value = self.monthly_start_value[month]
            if previous_value is not None:
                profit = current_value - previous_value
                monthly_pnl[previous_month] = {
                    "profit": profit,
                    "percentage": (profit / previous_value) 
                }
            previous_value = current_value
            previous_month = month

        # 計算每年盈虧
        sorted_years = sorted(self.yearly_start_value.keys())  # 按時間排序
        previous_value = None
        previous_year = None
        yearly_pnl = {}
        for year in sorted_years:
            current_value = self.yearly_start_value[year]
            if previous_value is not None:
                profit = current_value - previous_value
                yearly_pnl[previous_year] = {
                    "profit": profit,
                    "percentage": (profit / previous_value) 
                }
            previous_value = current_value
            previous_year = year

        self.rets["monthly_pnl"] = monthly_pnl
        self.rets["yearly_pnl"] = yearly_pnl


In [394]:
class GridTradingStrategy(bt.Strategy):
    params = (
        ('grid_size', 100),
        ('order_size', 1),
    )

    def __init__(self):
        self.last_price = None
        self.monthly_start_cash = {}  # 紀錄每個月的起始資金
        self.current_month = None  # 紀錄當前月份

    def next(self):
        if self.position:
            self.sell(size=1)
        self.buy(size=1)
        
                

        # 網格交易策略
        # if self.last_price is None:
        #     self.last_price = self.data.close[0]
        #     return

        # price_difference = self.data.close[0] - self.last_price

        # if price_difference >= self.params.grid_size:
        #     self.sell(size=self.params.order_size)
        #     self.last_price = self.data.close[0]
        # elif price_difference <= -self.params.grid_size:
        #     self.buy(size=self.params.order_size)
        #     self.last_price = self.data.close[0]
    def stop(self):
        pass
        # self.close()


In [395]:
test_data = {
    'datetime': pd.to_datetime(['2024-01-01', '2024-01-05','2024-02-01', '2024-02-05']),  # 每月一筆
    'open': [40000, 41000, 42000, 43000],
    'high': [40500, 41500, 40500, 43500],
    'low': [39500, 40500, 39500, 42500],
    'close': [40200, 41300, 42400, 43500],
    'volume': [1000, 1200, 1000, 1500],
}
test_data = pd.DataFrame(test_data)
test_data.set_index('datetime', inplace=True)
test_data = bt.feeds.PandasData(dataname=test_data)

In [396]:
cerebro = bt.Cerebro()
cerebro.addstrategy(GridTradingStrategy)
cerebro.adddata(data)
cerebro.addanalyzer(PNLAnalyzer, _name='pnl_analyzer')

# 設定初始資金
start_cash = 100000
cerebro.broker.set_cash(start_cash)

print(f'Start Portfolio Value: {cerebro.broker.getvalue()}')
results = cerebro.run()
print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

# 取得 Analyzer 結果
analyzer = results[0].analyzers.pnl_analyzer.get_analysis()

# 輸出每月、每年盈虧報告
print("\n==== 每月盈虧報告 ====")
for month, data in analyzer['monthly_pnl'].items():
    print(f"{month} 盈虧: {data['profit']:.2f}，百分比: {data['percentage']*100:.2f}%")
print("\n==== 每年盈虧報告 ====")
for year, data in analyzer['yearly_pnl'].items():
    print(f"{year} 盈虧: {data['profit']:.2f}，百分比: {data['percentage']*100:.2f}%")

Start Portfolio Value: 100000
Final Portfolio Value: 193544.91999999975

==== 每月盈虧報告 ====
2017-08 盈虧: 549.83，百分比: 0.55%
2017-09 盈虧: -456.43，百分比: -0.45%
2017-10 盈虧: 2375.50，百分比: 2.37%
2017-11 盈虧: 4029.01，百分比: 3.93%
2017-12 盈虧: 2597.01，百分比: 2.44%
2018-01 盈虧: -4155.48，百分比: -3.81%
2018-02 盈虧: 1695.48，百分比: 1.62%
2018-03 盈虧: -4106.99，百分比: -3.85%
2018-04 盈虧: 2258.47，百分比: 2.20%
2018-05 盈虧: -1550.47，百分比: -1.48%
2018-06 盈虧: -1164.20，百分比: -1.13%
2018-07 盈虧: 1247.77，百分比: 1.22%
2018-08 盈虧: -404.57，百分比: -0.39%
2018-09 盈虧: -588.40，百分比: -0.57%
2018-10 盈虧: -201.61，百分比: -0.20%
2018-11 盈虧: -2219.98，百分比: -2.17%
2018-12 盈虧: -392.88，百分比: -0.39%
2019-01 盈虧: -335.07，百分比: -0.34%
2019-02 盈虧: 360.93，百分比: 0.36%
2019-03 盈虧: 321.56，百分比: 0.32%
2019-04 盈虧: 1238.64，百分比: 1.24%
2019-05 盈虧: 3160.87，百分比: 3.13%
2019-06 盈虧: 2080.86，百分比: 2.00%
2019-07 盈虧: -249.94，百分比: -0.24%
2019-08 盈虧: -650.01，百分比: -0.61%
2019-09 盈虧: -1432.54，百分比: -1.36%
2019-10 盈虧: 939.17，百分比: 0.90%
2019-11 盈虧: -1840.72，百分比: -1.75%
2019-12 盈虧: -190.04，百分比:

## Export Report to Excel

In [397]:
import xlsxwriter

In [398]:
# 創建 Excel 檔案
wb = xlsxwriter.Workbook('Results.xlsx')
pnl_sheet = wb.add_worksheet("PNL")

In [399]:
# 設定格式
font_format_title = wb.add_format({
    'font_name': 'Calibri (Body)',
    'font_size': 15,
    'bold': False,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True
})

font_format_positive_value = wb.add_format({
    'font_name': 'Calibri (Body)',
    'font_size': 13,
    'bold': False,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True
})

font_format_negative_value = wb.add_format({
    'font_name': 'Calibri (Body)',
    'font_size': 11,
    'font_color': 'red',
    'bold': False,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True
})

font_format_positive_percentage = wb.add_format({
    'font_name': 'Calibri (Body)',
    'font_size': 13,
    'bg_color': '#e8e8e8',
    'bold': False,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True,
    'num_format': '0.00%'  # 這裡設置顯示為百分比，並保留兩位小數
})

font_format_negative_percentage = wb.add_format({
    'font_name': 'Calibri (Body)',
    'font_size': 11,
    'font_color': 'red',
    'bg_color': '#e8e8e8',
    'bold': False,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True,
    'num_format': '0.00%'  # 這裡設置顯示為百分比，並保留兩位小數
})

In [400]:
# 1️⃣ **寫入年份 (B1, C1, D1, ...)**
col = 1  # 從 B 欄開始
years = sorted(analyzer['yearly_pnl'].keys())  # 確保按時間排序
for year in years:
    pnl_sheet.write(0, col, year, font_format_title)  # 年份標題寫在第一行
    col += 1  # 右移一欄
    pnl_sheet.write(0, col, year, font_format_title)  # 年份標題寫在第一行
    col += 1  # 右移一欄

# 2️⃣ **寫入月份 (A2, A3, ..., A13)**
months = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
row = 1  # 從第 2 行開始
for month in months:
    pnl_sheet.write(row, 0, month, font_format_title)  # 月份標題寫在第一列
    row += 1

# 3️⃣ **寫入盈虧數據**
for year_idx, year in enumerate(years):
    col = year_idx * 2 + 1  # 年份欄位 (B, C, D, ...)
    for month_idx, month in enumerate(months):
        month_key = f"{year}-{str(month_idx+1).zfill(2)}"  # e.g., '2024-01'
        if month_key in analyzer['monthly_pnl']:  # 確保有這個月份的數據
            profit = analyzer['monthly_pnl'][month_key]['profit']
            percentage = analyzer['monthly_pnl'][month_key]['percentage']
            pnl_sheet.write(month_idx + 1, col, profit, font_format_positive_value if profit >= 0 else font_format_negative_value)
            pnl_sheet.write(month_idx + 1, col + 1, percentage, font_format_positive_percentage if percentage >= 0 else font_format_negative_percentage)

# 4️⃣ **填入年報酬、年DD、總獲利**
pnl_sheet.write('A14', '年報酬', font_format_title)
pnl_sheet.write('A15', '年DD', font_format_title)
pnl_sheet.write('D15', '最大策略虧損', font_format_title)
pnl_sheet.write('G15', '總獲利金額', font_format_title)
pnl_sheet.write('J15', '總報酬比例', font_format_title)

# 5️⃣ **填入年報酬數據**
row = 13  # A14 對應的 row index
for year_idx, year in enumerate(years):
    col = year_idx * 2 + 1  # 年份欄位
    profit = analyzer['yearly_pnl'][year]['profit']
    percentage = analyzer['yearly_pnl'][year]['percentage']
    pnl_sheet.write(row, col, profit, font_format_positive_value if profit >= 0 else font_format_negative_value)
    pnl_sheet.write(row, col+1, percentage, font_format_positive_percentage if percentage >= 0 else font_format_negative_percentage)

# 6️⃣ **填入年DD**

# 7️⃣ **填入最大策略虧損**

# 8️⃣ **填入總獲利金額**
total_profit = cerebro.broker.getvalue() - start_cash
pnl_sheet.merge_range('H15:I15', total_profit, font_format_positive_value if total_profit >= 0 else font_format_negative_value)

# 9️⃣ **填入總報酬比例**
total_percentage = total_profit / start_cash
pnl_sheet.merge_range('K15:L15', total_percentage, wb.add_format({'font_name': 'Calibri (Body)', 'font_size': 13 if total_percentage >= 0 else 11, 'align': 'center', 'valign': 'vcenter', 'text_wrap': True, 'num_format': '0.00%', 'color': 'black' if total_percentage >= 0 else 'red'}))



# 存檔並關閉
wb.close()
